In [1]:
import glob
import os

import numpy as np
from utility_programs.read_routines import SAMI

from scipy.io import netcdf_file

In [31]:
ds_full = SAMI.auto_read('/glade/u/home/abukowski/scratch/simstorm-20110521/GITM-simstorm-run1/sami-gitm-coupled/', cols='edens', split_by_time=True, end_idx=1)

['/glade/u/home/abukowski/scratch/simstorm-20110521/GITM-simstorm-run1/sami-gitm-coupled/t110521_021500.nc']


In [32]:
coord_ds = ds_full[['glon','glat','alt']]
coord_ds = coord_ds.drop_vars(['time','mlat','mlon','malt']).load().compute()
coord_ds

<xarray.Dataset>
Dimensions:  (nlt: 80, nf: 72, nz: 256)
Coordinates:
    glon     (nlt, nf, nz) float32 290.3 290.3 290.3 290.3 ... 277.6 277.6 277.6
    glat     (nlt, nf, nz) float32 -13.74 -13.74 -13.74 ... 80.04 80.04 80.04
    alt      (nlt, nf, nz) float32 80.02 80.03 80.05 80.06 ... 89.94 87.09 84.4
Dimensions without coordinates: nlt, nf, nz
Data variables:
    *empty*

In [33]:
coord_ds

<xarray.Dataset>
Dimensions:  (nlt: 80, nf: 72, nz: 256)
Coordinates:
    glon     (nlt, nf, nz) float32 290.3 290.3 290.3 290.3 ... 277.6 277.6 277.6
    glat     (nlt, nf, nz) float32 -13.74 -13.74 -13.74 ... 80.04 80.04 80.04
    alt      (nlt, nf, nz) float32 80.02 80.03 80.05 80.06 ... 89.94 87.09 84.4
Dimensions without coordinates: nlt, nf, nz
Data variables:
    *empty*

In [34]:
ds_full

<xarray.Dataset>
Dimensions:         (nlt: 80, nf: 72, nz: 256)
Coordinates:
    time            datetime64[ns] ...
    mlat            (nlt, nf, nz) float32 ...
    mlon            (nlt, nf, nz) float32 ...
    malt            (nlt, nf, nz) float32 ...
    glat            (nlt, nf, nz) float32 -13.74 -13.74 -13.74 ... 80.04 80.04
    glon            (nlt, nf, nz) float32 290.3 290.3 290.3 ... 277.6 277.6
    alt             (nlt, nf, nz) float32 80.02 80.03 80.05 ... 89.94 87.09 84.4
Dimensions without coordinates: nlt, nf, nz
Data variables: (12/25)
    edens           (nlt, nf, nz) float64 ...
    h+dens          (nlt, nf, nz) float64 ...
    o+dens          (nlt, nf, nz) float64 ...
    n+odens         (nlt, nf, nz) float64 ...
    o2+dens         (nlt, nf, nz) float64 ...
    he+dens         (nlt, nf, nz) float64 ...
    ...              ...
    h+vel_parallel  (nlt, nf, nz) float64 ...
    o+vel_parallel  (nlt, nf, nz) float64 ...
    mer_exb         (nlt, nf, nz) float64 ...
    zon_exb         (nlt, nf, nz) float64 ...
    zon_neut        (nlt, nf, nz) float64 ...
    mer_neut        (nlt, nf, nz) float64 ...

In [6]:
lat_raw = coord_ds.glat.values
lon_raw = coord_ds.glon.values
alt_raw = coord_ds.alt.values


# grid = {'malt':ds_full.malt.values,
#         'mlat':ds_full.mlat.values,
#         'mlon':ds_full.mlon.values}

# dtime =  pd.Timestamp(ds_full.time.values)

del coord_ds, ds_full

In [7]:
nlt = 80
nf = 72
nz = 256

In [8]:
nodelon = []
nodelat = []
height = []
nodeid = np.zeros([nlt,nf,nz])

# grid2 = {'malt':[],'mlat':[],'mlon':[]}

counter=0

for lt in range(nlt):
    for f in range(nf):
        for z in range(nz):
            nodelon.append(lon_raw[lt, f, z])
            nodelat.append(lat_raw[lt, f, z])
            height.append(alt_raw[lt, f, z])
            
            # grid2['malt'].append(grid['malt'][lt, f, z])
            # grid2['mlat'].append(grid['mlat'][lt, f, z])
            # grid2['mlon'].append(grid['mlon'][lt, f, z])

            
            nodeid[lt, f, z] = counter
            counter += 1

In [9]:
# carts = grid_to_cartesian(grid2, dtime)
# carts = np.array([carts.x, carts.y, carts.z]).T
# carts.shape

In [10]:
counter, nz * nf * nlt

(1474560, 1474560)

In [11]:
vert_ids = []

didnt_work = []

known_bad = []
not_bad = []

weird = []
weird2 = []


for lt in range(nlt):
    for f in range(nf):
        for z in range(nz):
            
            if lt < nlt-1 and f != nf-1 and z != 0:
                try:
                    vert_ids.append([nodeid[lt, f, z-1],
                                     nodeid[lt, f, z-1],
                                     nodeid[lt+1, f, z],
                                     nodeid[lt+1, f, z],
                                     nodeid[lt, f+1, z],
                                     nodeid[lt, f+1, z],
                                     nodeid[lt+1, f+1, z-1],
                                     nodeid[lt+1, f+1, z-1],])
                    
                except IndexError:
                    didnt_work.append([lt,f,z])
                    
                except:
                    print(lt, f, z)
                    
            elif f != nf-1 and z != 0:
                try:
                    # pp+=1
                    vert_ids.append([nodeid[lt, f, z-1],
                                     nodeid[lt, f, z-1],
                                     nodeid[0, f, z],
                                     nodeid[0, f, z],
                                     nodeid[lt, f+1, z],
                                     nodeid[lt, f+1, z],
                                     nodeid[0, f+1, z-1],
                                     nodeid[0, f+1, z-1],])
                    
                except IndexError:
                    didnt_work.append([lt,f,z])
                    
                except:
                    print(lt, f, z)
            
            
            elif f == nf-1 or z == 0:
                try:
                    known_bad.append([lt, f, z])
                except IndexError:
                    weird.append([lt,f,z])
                except:
                    print(lt, f, z)
                    
            else:
                weird2.append([lt,f,z])

In [12]:
len(not_bad), len(known_bad)

(0, 26160)

In [13]:
len(didnt_work)

0

In [14]:
len(weird), len(weird2)

(0, 0)

In [15]:
#UGRID FORMAT

In [16]:
try:
    if ds.isopen():
        ds.close()
        ds = netcdf_file('/glade/u/home/abukowski/mesh-test/ugrid-mesh.nc', 'w')#, diskless = True)
    else:
        ds = netcdf_file('/glade/u/home/abukowski/mesh-test/ugrid-mesh.nc', 'w')#, diskless = True)
except:
    ds = netcdf_file('/glade/u/home/abukowski/mesh-test/ugrid-mesh.nc', 'w')#, diskless = True)
    
numnodes = ds.createDimension('nnodes', len(nodelon))
numcells = ds.createDimension('ncells', len(vert_ids))
eight = ds.createDimension('eight', 8)


meshattrs = ds.createVariable('mesh','i4', ())
meshattrs.cf_role = 'mesh_topology'
meshattrs.topology_dimension = 3.
meshattrs.node_coordinates = 'nodelon nodelat height'
meshattrs.volume_node_connectivity = 'vertids'
meshattrs.volume_shape_type = 'meshtype'

loncoords = ds.createVariable('nodelon','double',('nnodes',))
loncoords.standard_name = 'longitude'
loncoords.units = 'degrees_east'
loncoords[:] = nodelon

latcoords = ds.createVariable('nodelat','double',('nnodes',))
latcoords.standard_name = 'latitude'
latcoords.units = 'degrees_north'
latcoords[:] = nodelat

hcoords = ds.createVariable('height','double',('nnodes',))
hcoords.standard_name = 'elevation'
hcoords.units = 'kilometers'
hcoords[:] = nodelon

meshvals = ds.createVariable('vertids','i4',('ncells','eight'))
meshvals.cf_role = 'volume_node_connectivity'
# meshvals.start_index = 0.
meshvals[:] = vert_ids

meshtypes = ds.createVariable('meshtype','i4',('ncells',))
meshtypes.cf_role = "volume_shape_type" ;
meshtypes.flag_range = 1. 
meshtypes.flag_values = 1. 
meshtypes.flag_meanings = "hexahedron" 

meshtypes[:] = [1 for i in range(len(vert_ids))]


print(ds)

ds.sync()

ds.close()

In [17]:
#ESMF UNSTRUCT MESH FORMAT

In [19]:
#ugrid out

In [20]:
lons_out =  np.arange(0, 360, 5)
lats_out = np.arange(-90, 90, 2.0)
alts_out =  np.arange(250, 2000, 50)


nodelon_out = []
nodelat_out = []
height_out = []

nlons_out = len(lons_out)
nlats_out = len(lats_out)
nalts_out = len(alts_out)
nodeid_out = np.zeros([nlons_out, nlats_out, nalts_out])
counter_out = 0

for x in range(nlons_out):
    for y in range(nlats_out):
        for z in range(nalts_out):
            
            nodelon_out.append(lons_out[x])
            nodelat_out.append(lats_out[y])
            height_out.append(alts_out[z])
            
            nodeid_out[x,y,z] = counter_out
            counter_out +=1


vert_ids_out = []

didnt_work_out = []


known_bad_out = []
not_bad_out = []

weird_out = []
weird2_out = []
maxalts = []

what = []

for x in range(nlons_out):
    for y in range(nlats_out):
        for z in range(nalts_out):

            if x != nlons_out-1 and y != 0 and z != nalts_out-1:
                try:
                    vert_ids_out.append(
                        [nodeid_out[x,   y,   z],
                         nodeid_out[x,   y-1, z],
                         nodeid_out[x+1, y-1, z],
                         nodeid_out[x+1, y,   z],
                         nodeid_out[x,   y,   z+1],
                         nodeid_out[x,   y-1, z+1],
                         nodeid_out[x+1, y-1, z+1],
                         nodeid_out[x+1, y,   z+1],])
                except IndexError:
                    didnt_work_out.append([x,y,z])
                    
                except:
                    print(x, y, z)
                    
                    
            elif x == nlons_out-1 and y != 0 and z != nalts_out-1:
                try:
                    vert_ids_out.append([
                        nodeid_out[x, y,   z-1],
                        nodeid_out[x, y-1, z-1],
                        nodeid_out[0, y-1, z],
                        nodeid_out[0, y,   z],
                        nodeid_out[x, y,   z],
                        nodeid_out[x, y-1, z],
                        nodeid_out[0, y-1, z-1],  
                        nodeid_out[0, y,   z-1],])
                except IndexError:
                    didnt_work_out.append([x,y,z])
                    
                except:
                    print(x, y, z)
                    
            elif x != nlons_out-1 and y==0 and z!= nalts_out-1:
                try:
                    vert_ids_out.append([
                        nodeid_out[x,   y,   z-1],
                        nodeid_out[x,   -1,  z-1],
                        nodeid_out[x+1, -1,  z],
                        nodeid_out[x+1, y,   z],
                        nodeid_out[x,   y,   z],
                        nodeid_out[x,   -1,  z],
                        nodeid_out[x+1, -1,  z-1],  
                        nodeid_out[x+1, y,   z-1],])
                except IndexError:
                    didnt_work_out.append([x,y,z])
                    
                except:
                    print(x, y, z)
                    
            elif x == nlons_out-1 and y==0 and z!= nalts_out-1:
                try:
                    vert_ids_out.append([
                        nodeid_out[x, y,   z-1],
                        nodeid_out[x, -1,  z-1],
                        nodeid_out[0, -1,  z],
                        nodeid_out[0, y,   z],
                        nodeid_out[x, y,   z],
                        nodeid_out[x, -1,  z],
                        nodeid_out[0, -1,  z-1],  
                        nodeid_out[0, y,   z-1],])
                except IndexError:
                    didnt_work_out.append([x,y,z])
                    
                except:
                    print(x, y, z)
                    
            elif z!= nalts_out-1:
                weird2_out.append([x,y,z])
                
            elif z == nalts_out-1:
                maxalts.append([x,y,z])
                
            else:
                what.append([x,y,z])
                

# for x in range(nlats_out):
#     for y in range(nlons_out):
#         for z in range(nalts_out):

#             if x < (nlats_out-1) and y != nlons_out-1 and z != nalts_out-1:
#                 try:
#                     vert_ids.append([nodeid_out[x, y, z-1],
#                                      nodeid_out[x, y, z-1],
#                                      nodeid_out[x+1, y, z],
#                                      nodeid_out[x+1, y, z],
#                                      nodeid_out[x, y+1, z],
#                                      nodeid_out[x, y+1, z],
#                                      nodeid_out[x+1, y+1, z-1],
#                                      nodeid_out[x+1, y+1, z-1],])
                    
#                 except IndexError:
#                     didnt_work_out.append([x,y,z])
                    
#                 except:
#                     print(x, y, z)
                    
#             elif y != 71 and z != 0:
#                 try:
#                     vert_ids_out.append([nodeid_out[x, y, z-1],
#                                      nodeid_out[x, y, z-1],
#                                      nodeid_out[0, y, z],
#                                      nodeid_out[0, y, z],
#                                      nodeid_out[x, y+1, z],
#                                      nodeid_out[x, y+1, z],
#                                      nodeid_out[0, y+1, z-1],
#                                      nodeid_out[0, y+1, z-1],])
#                 except IndexError:
#                     didnt_work_out.append([x,y,z])
                    
#                 except:
#                     print(x, y, z)
            
            
#             elif y == nlons_out-1 or z == 0:
#                 try:
#                     known_bad_out.append([x, y, z])
#                 except IndexError:
#                     weird_out.append([x,y,z])
#                 except:
#                     print(x, y, z)
                    
#             else:
#                 weird2_out.append([x,y,z])

In [21]:
len(not_bad_out), len(didnt_work_out), len(weird_out), len(weird2_out)

(0, 0, 0, 0)

In [22]:
nalts_out*nlons_out*nalts_out

88200

In [23]:
len(lons_out), len(lats_out), nlons_out, nlats_out, 

(72, 90, 72, 90)

In [24]:
nlons_out * nlats_out

6480

In [25]:
nlons_out

72

In [26]:
len(maxalts), len(what)

(6480, 0)

In [27]:
len(vert_ids_out), len(nodelon_out), len(vert_ids_out)/len(nodelon_out)

(220320, 226800, 0.9714285714285714)

In [28]:
len(vert_ids), len(nodelon), len(vert_ids)/ len(nodelon)

(1448400, 1474560, 0.9822591145833334)

In [29]:
print('hi')

hi


In [30]:
try:
    if ds_out.isopen():
        ds_out.close()
        ds_out = netcdf_file('/glade/u/home/abukowski/mesh-test/ugrid-mesh_out.nc', 'w')#, diskless = True)
    else:
        ds_out = netcdf_file('/glade/u/home/abukowski/mesh-test/ugrid-mesh_out.nc', 'w')#, diskless = True)
except:
    ds_out = netcdf_file('/glade/u/home/abukowski/mesh-test/ugrid-mesh_out.nc', 'w')#, diskless = True)
    
numnodes_out = ds_out.createDimension('nnodes', len(nodelon_out))
numcells_out = ds_out.createDimension('ncells', len(vert_ids_out))
eight_out = ds_out.createDimension('eight', 8)


meshattrs_out = ds_out.createVariable('mesh','i4', ())
meshattrs_out.cf_role = 'mesh_topology'
meshattrs_out.topology_dimension = 3.
meshattrs_out.node_coordinates = 'nodelon nodelat height'
meshattrs_out.volume_node_connectivity = 'vertids'
meshattrs_out.volume_shape_type = 'meshtype'

loncoords_out = ds_out.createVariable('nodelon','double',('nnodes',))
loncoords_out.standard_name = 'longitude'
loncoords_out.units = 'degrees_east'
loncoords_out[:] = nodelon_out

latcoords_out = ds_out.createVariable('nodelat','double',('nnodes',))
latcoords_out.standard_name = 'latitude'
latcoords_out.units = 'degrees_north'
latcoords_out[:] = nodelat_out

hcoords_out = ds_out.createVariable('height','double',('nnodes',))
hcoords_out.standard_name = 'elevation'
hcoords_out.units = 'kilometers'
hcoords_out[:] = nodelon_out

meshvals_out = ds_out.createVariable('vertids','i4',('ncells','eight'))
meshvals_out.cf_role = 'volume_node_connectivity'
# meshvals.start_index = 0.
meshvals_out[:] = vert_ids_out

meshtypes_out = ds_out.createVariable('meshtype','i4',('ncells',))
meshtypes_out.cf_role = "volume_shape_type" ;
meshtypes_out.flag_range = 1. 
meshtypes_out.flag_values = 1. 
meshtypes_out.flag_meanings = "hexahedron" 

meshtypes_out[:] = [1 for i in range(len(vert_ids_out))]


print(ds_out)

ds_out.sync()

ds_out.close()